In [1]:
import pandas as pd
import scanpy as sc
import os
import numpy as np

In [2]:
datadir = '/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/'
os.chdir(datadir)

In [3]:
mtx =os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI.mtx")
cells=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI_cells.txt"),header=None,index_col=0)
features=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/','sct_corrected_UMI_genes.txt'),header=None,sep='\t',index_col=0)
adata = sc.read_mtx(mtx)


In [6]:
adata.obs['CellID']= cells.index.tolist()
adata.var['Gene']= features.index.tolist()
adata.var.index= adata.var['Gene']
display(adata)
print(f'Max val of sct matrix before transformation, {np.max(adata.X)}')

AnnData object with n_obs × n_vars = 15954 × 19647
    obs: 'CellID'
    var: 'Gene'

Max val of sct matrix before transformation, 8.334712028503418


### Merging converted human genes

In [7]:
m_h_df = pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/','blca_publication_OUTPUT_m_h.csv'),index_col=0)
display(m_h_df)

,mouse_gene,orig.id,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name
Xkr4,Xkr4,Xkr4,ENSG00000206579,XKR4
Sox17,Sox17,Sox17,ENSG00000164736,SOX17
Mrpl15,Mrpl15,Mrpl15,ENSG00000137547,MRPL15
Lypla1,Lypla1,Lypla1,ENSG00000120992,LYPLA1
Gm37988,Gm37988,Gm37988,NaN,NaN
...,...,...,...,...
Dmrt3,Dmrt3,Dmrt3,ENSG00000064218,DMRT3
Lipo2,Lipo2,Lipo2,NaN,NaN
Gm47936,Gm47936,Gm47936,NaN,NaN
Fgf8,Fgf8,Fgf8,ENSG00000107831,FGF8


In [11]:
for i in adata.var.index.intersection(m_h_df.index.to_list()):
    adata.var.loc[i,'Human_gene'] = m_h_df.loc[i,'hsapiens_homolog_associated_gene_name']
display(adata.var)

,Gene,Human_gene
Gene,,
Xkr4,Xkr4,XKR4
Sox17,Sox17,SOX17
Mrpl15,Mrpl15,MRPL15
Lypla1,Lypla1,LYPLA1
Gm37988,Gm37988,NaN
...,...,...
Gm47936,Gm47936,NaN
Gm29595,Gm29595,NaN
Fgf8,Fgf8,FGF8


### Passing annotations

In [12]:
adata.obs['Sample'] = adata.obs['CellID'].str.split('_').str[0]
adata.obs.index = adata.obs['CellID']

In [13]:
adata.obs['Type'] = 'N/A'
for i in ['GSM5288668', 'GSM5288669']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'NMIBC'
for i in ['GSM5288670', 'GSM5288671']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'MIBC'    
for i in ['GSM5288672', 'GSM5288674']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'Healthy'    

In [14]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata,base=2)
print(f'Max val after log1p, {np.max(adata.to_df())}')


Max val after log1p, 9.472813606262207


### Subset using Lincs landmark genes 

In [16]:
gene_info_df=pd.read_csv('/home/jing/Phd_project/project_GBM/gbm_DATA/gbm_DATA_LINCS/2020/info/geneinfo_beta.txt', sep = "\t", index_col = 1)

gene_info_df = gene_info_df.sort_values(by = ["gene_symbol"])
display(gene_info_df)

,gene_id,ensembl_id,gene_title,gene_type,src,feature_space
gene_symbol,,,,,,
A1CF,29974,ENSG00000148584,APOBEC1 complementation factor,protein-coding,NCBI,inferred
A2M,2,ENSG00000175899,alpha-2-macroglobulin,protein-coding,NCBI,best inferred
A4GALT,53947,ENSG00000128274,"alpha 1,4-galactosyltransferase (P blood group)",protein-coding,NCBI,best inferred
A4GNT,51146,ENSG00000118017,"alpha-1,4-N-acetylglucosaminyltransferase",protein-coding,NCBI,inferred
AAAS,8086,ENSG00000094914,aladin WD repeat nucleoporin,protein-coding,NCBI,best inferred
...,...,...,...,...,...,...
ZXDB,158586,ENSG00000198455,zinc finger X-linked duplicated B,protein-coding,NCBI,inferred
ZXDC,79364,ENSG00000070476,ZXD family zinc finger C,protein-coding,NCBI,best inferred
ZYX,7791,ENSG00000159840,zyxin,protein-coding,NCBI,best inferred


In [20]:
gene_info_df[gene_info_df['feature_space'] =='landmark'].index

Index(['AARS', 'ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1',
       'ACAA1', 'ACAT2',
       ...
       'ZMIZ1', 'ZMYM2', 'ZNF131', 'ZNF274', 'ZNF318', 'ZNF395', 'ZNF451',
       'ZNF586', 'ZNF589', 'ZW10'],
      dtype='object', name='gene_symbol', length=978)

In [25]:
human_gene_set = set(adata.var['Human_gene'].to_list())
landmark_gene_set = set(gene_info_df[gene_info_df['feature_space'] == 'landmark'].index.to_list())

intersected_genes = human_gene_set.intersection(landmark_gene_set)


In [30]:
m_h_conversion = pd.read_csv('/home/jing/Phd_project/project_UCD_blca/blca_OUTPUT/blca_OUTPUT_m_h_convert/m_h_convertion.csv',index_col=0)

In [ ]:
for i in adata.var.index.intersection(landmark_gene_set):
    adata.var.loc[i,'pre_m_h'] = m_h_df.loc[i,'hsapiens_homolog_associated_gene_name']

In [54]:
adata.var

,Gene,Human_gene
Gene,,
Xkr4,Xkr4,XKR4
Sox17,Sox17,SOX17
Mrpl15,Mrpl15,MRPL15
Lypla1,Lypla1,LYPLA1
Gm37988,Gm37988,NaN
...,...,...
Gm47936,Gm47936,NaN
Gm29595,Gm29595,NaN
Fgf8,Fgf8,FGF8


In [43]:
len(set(m_h_conversion['Mouse_Genes'].to_list()).intersection(set(adata.var['Gene'].to_list())))

952

In [53]:
len(set(m_h_conversion['Human_Genes'].to_list()).intersection(landmark_gene_set))

965

In [46]:
m_h_conversion['Mouse_Genes'].to_list()).intersection(set(adata.var['Gene'].to_list()).intersection(landmark_gene_set)


SyntaxError: unmatched ')' (2000699833.py, line 1)

In [ ]:

# Subset adata to only these intersected genes
adata_subset = adata[:, adata.var['Human_gene'].isin(intersected_genes)].copy()


In [ ]:
#For raw data retrival 
def subset(data,name):
    mtx =f"{data}_filtered_matrix.mtx.gz"
    cells=pd.read_csv(f"{data}_filtered_barcodes.tsv.gz",header=None)
    features=pd.read_csv(f'{data}_filtered_features.tsv.gz',header=None,sep='\t')
    adata = sc.read_mtx(mtx)
    
    adata= adata.T
    print(adata)
    adata.obs['CellID']= cells[0].tolist()
    adata.var['Gene']= features[1].tolist()
    adata.var.index= adata.var['Gene']
    
    adata.var_names_make_unique()
    barcodes_subset = barcodes_uro.loc[barcodes_uro['Sample'] == name, 'Barcode']
    adata_subset = adata[adata.obs['CellID'].isin(barcodes_subset)]
    print(adata_subset)
    adata_subset = adata_subset[:, adata_subset.var_names.isin(gene_list)]
    sc.pp.normalize_total(adata_subset, target_sum=1e4)
    sc.pp.log1p(adata_subset,base=2)
    adata_subset_df=adata_subset.to_df()
   # adata_subset_df.to_pickle(f"{name}_sub.pkl")
